In [1]:
import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt

import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

ImportError: Traceback (most recent call last):
  File "/home/groot/Websites/note/local/lib/python2.7/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/groot/Websites/note/local/lib/python2.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/groot/Websites/note/local/lib/python2.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
ImportError: libcuda.so.1: cannot open shared object file: No such file or directory


Failed to load the native TensorFlow runtime.

See https://www.tensorflow.org/install/install_sources#common_installation_problems

for some common reasons and solutions.  Include the entire stack trace
above this error message when asking for 

In [ ]:
tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows =10
pd.options.display.float_format = '{:.1f}'.format

In [ ]:
df=pd.read_csv("https://storage.googleapis.com/mledu-datasets/california_housing_train.csv", sep=",")

df=df.reindex(np.random.permutation(df.index))

df["median_house_value"] /= 1000
df.head()

In [ ]:
df.describe()

## Step 1: Feature and Feature Column aka predictor

1. ** feature_data **
* ** feature_column aka description ** 

In [ ]:
feature_data   = df[["total_rooms"]]
feature_column = [tf.feature_column.numeric_column("total_rooms")] 

## Step 2: Set target

In [ ]:
target = df["median_house_value"]

## Step 3: Linear regression configs here

1. optimizer : Type
* optimizer : clip_gradient

2. linear regressor: tf.estimator( feature_column , optimizer from 1)
    

In [ ]:
opt=tf.train.GradientDescentOptimizer(learning_rate=0000001)

opt=tf.contrib.estimator.clip_gradients_by_norm(opt, 5.0)

linear_regressor=tf.estimator.LinearRegressor(
    
    feature_columns= feature_column, 
    optimizer=opt
)

## Step 4: Make Input Function

In [ ]:
def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):

    features={key : np.array(value) for key, value in dict(features).items() }

# dataset opertions 
    
    ds=Dataset.from_tensor_slices( (features, targets)  )
    ds=ds.batch(batch_size).repeat(num_epochs)
#     Shuffle datset

    if shuffle:
        ds=ds.shuffle(buffer_size=10000)
        
    features, labels = ds.make_one_shot_iterator().get_next()
    return features, labels


## Step 5: Training 

In [ ]:
_ =linear_regressor.train(
input_fn = lambda: my_input_fn(feature_data, target),
    steps=100
)

## Step 6: Evaluate Model

In [ ]:
prediction_input_fn = lambda: my_input_fn(feature_data, target, num_epochs =1, shuffle=False)

predictions = linear_regressor.predict(input_fn=prediction_input_fn)

predictions =np.array(  
                        [item["predictions"][0] for item in predictions]
                    )

mse= metrics.mean_squared_error(predictions, target)
rmse= math.sqrt(mse)

print("MSE : %0.3f" %mse )
print("RMSE: %0.3f" %rmse)

#### Compare MSE and RMSE with range of data

In [ ]:
min_v= df["median_house_value"].min()
max_v= df["median_house_value"].max()
range_v= max_v - min_v

print("MIN  : %0.3f" %min_v )
print("MAX  : %0.3f" %max_v)
print("RANGE: %0.3f" %range_v )
print("RMSE : %0.3f" %rmse)

### Calibrating dataset

In [ ]:
cal_df=pd.DataFrame()
cal_df["predictions"]=pd.Series(predictions)
cal_df["target"]=pd.Series(target)
cal_df.describe()

#### Sampling

In [ ]:
sample=df.sample(n=300)

####  Ploting:

In [ ]:
x_0 = sample["total_rooms"].min()
x_1 = sample["total_rooms"].min()

weight = linear_regressor.get_variable_value('linear/linear_model/total_rooms/weights')[0]
bias   = linear_regressor.get_variable_value('linear/linear_model/bias_weights')

y_0 = weight * x_0 + bias
y_1 = weight * x_1 + bias

plt.plot([x_0, x_1], [y_0, y_1], c='r')

plt.ylabel("median_house_value")
plt.xlabel("total_rooms")

plt.scatter(sample["total_rooms"], sample["median_house_value"])

plt.show()

## Creating Library for our process

In [ ]:
def train_model(learning_rate, steps, batch_size, input_feature="total_rooms"):
    
    periods=10
    steps_per_period= steps / periods
    
    my_feature=input_feature
    my_feature_data = df[[my_feature]]
    my_label= "median_house_value"
    targets= df[my_label]
    
    # CReate feature columns     
    feature_columns= [tf.feature_column.numeric_column(my_feature)]
    
    # CReate input functions
    training_input_fn = lambda: my_input_fn(
                                            my_feature_data,
                                            targets,
                                            batch_size=batch_size)
    predictions_input_fn = lambda:my_input_fn(my_feature_data, 
                                              targets, 
                                              num_epochs=1, 
                                              shuffle= False)  
    
    # Create Linear Regressor
    opt=tf.train.GradientDescentOptimizer(learning_rate=0.00001)
    opt=tf.contrib.estimator.clip_gradients_by_norm(opt, 5.0)

    linear_regressor=tf.estimator.LinearRegressor(
        feature_columns =feature_columns,
        optimizer=opt
        )
    
    # Set up plotting      
    plt.figure(figsize=(15,6))
    plt.subplot(1,2,1)
    plt.title("Learned Line By Period")
    plt.ylabel(my_label)
    plt.xlabel(my_feature)
    sample=df.sample(n=300)
    plt.scatter(sample[my_feature], sample[my_label])
    colors = [ cm.coolwarm(x) 
              for x in np.linspace(-1, 1, periods)
             ]

    # Training model in loop
    print "TRainging model...."
    print "RMSE on training data..."
    
    rmses=[]
    
    for period in range (0, periods):
        linear_regressor.train(
            input_fn=training_input_fn,
            steps=steps_per_period
        )
        
        predictions =linear_regressor.predict(input_fn=prediction_input_fn)
        predictions =np.array([
                                item["predictions"][0] 
                                for item in predictions
                                ])
        # Compute Loss      
        rmse = math.sqrt(
            metrics.mean_squared_error(predictions, targets)
            )
        # Print current loss
        print " period %02d; %0.2f" % (period, rmse)
    
        # Add to our list
        rmses.append(rmse)
    
        # Starting Plots         
        y_extents =np.array([0, sample[my_label].max()])
        
        weight = linear_regressor.get_variable_value('linear/linear_model/total_rooms/weights')[0]
        bias   = linear_regressor.get_variable_value('linear/linear_model/bias_weights')
        
        x_extents = (y_extents - bias) / weight
        x_extents = np.maximum(np.minimum(x_extents,
                                      sample[my_feature].max()),
                           sample[my_feature].min())
        y_extents = weight * x_extents + bias
        plt.plot(x_extents, y_extents, color=colors[period]) 
    
    print "Modeling done"
    
    # Start Plotting
    plt.subplot(1,2,2)
    plt.ylabel("RMSEs")
    plt.xlabel("Periods")
    plt.title("RMSEs vs Periods")
    plt.tight_layout()
    plt.plot(rmses)
    

    # Output Table calibrated 
    calibration_data = pd.DataFrame()
    calibration_data["predictions "]=pd.Series(predictions)
    calibration_data["targets"] =pd.Series(targets)
    display.display(calibration_data.describe())
    
    print "Final RMSE : %0.2f" % rmse

In [ ]:
train_model(
    learning_rate=0.000005,
    steps=200,
    batch_size =15
    )
